In [27]:
#UNSUPERVISED LEARNING FOR THE 2ND DATASET
import numpy as np
import pandas as pd

In [28]:
#DATA FRAME FOR DATASET
df = pd.read_csv('amazon_reviews2.csv',sep=',')

In [29]:
df.columns

Index(['url', 'review_bold', 'ratings', 'review', 'verified', 'date', 'by',
       'profile_id', 'most_rev', 'by_link', 'helpful', 'product',
       'product_link', 'review_sentiment'],
      dtype='object')

In [30]:
#REVIEW_BOLD IS THE REVIEW TITLE
df.rename(columns={'review_bold':'review_title'}, inplace=True)

In [31]:
#BY IS USER_NAME
df.rename(columns={'by':'user_name'}, inplace=True)

#profile_id is user_id
df.rename(columns={'profile_id':'user_id'}, inplace=True)

#most_rev is highest number of reviews made in a day by user
df.rename(columns={'most_rev':'max_reviews_day'}, inplace=True)

In [32]:
#THEY USE THE TEXTBLOB SENTIMENT SCORE

#helpful is number of helpful votes
df.rename(columns={'helpful':'helpful_votes'}, inplace=True)

In [10]:
#LESS VARIETY IN PRODUCTS THAN PREVIOUS DATASET
df['product'].value_counts()

In Ear Earphones Black                                                                                                                                                                                                            115
Earphones In ear Black&Red                                                                                                                                                                                                        115
Earphones, XuDirect Earbuds In-line Control Headphones with Microphone Black                                                                                                                                                      107
AmazonBasics Sport In-Ear Headphones                                                                                                                                                                                               83
Earphones In Line Headphones                                                    

In [33]:
#ADD A NUMBER OF REVIEWS COLUMN [PRODUCT]
num_reviews = df.groupby('product').size()

df['total_reviews'] = df['product'].map(num_reviews)

In [45]:
#ADD A NUMBER OF REVIEWS BY USER COLUMN
num_reviews = df.groupby('user_id').size()
df['total_user_reviews'] = df['user_id'].map(num_reviews)

In [38]:
#ADD AN AVERAGE-RATING COLUMN
average_ratings = df.groupby('product')['ratings'].mean()

df['average_rating'] = df['product'].map(average_ratings)

In [39]:
#ADD RATING DEVIATION COLUMN
df['rating_deviation'] = df['ratings'] - df['average_rating']

In [42]:
#ADD REVIEW LENGTH COLUMN
df['review_length'] = df['review'].apply(lambda d: len(d))

In [43]:
#ADD TITLE LENGTH COLUMN
df['title_length'] = df['review_title'].apply(lambda d: len(d))

In [52]:
#ADD TIME DIFFERENCE BETWEEN FIRST REVIEW OF USER AND LAST REVIEW OF USER COLUMN

# Step 1: Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d %B %Y')

# Step 2: Group the DataFrame by user_id and calculate the date difference
date_diff = df.groupby('user_id')['date'].apply(lambda x: (x.max() - x.min()).days).reset_index()

# Step 3: Merge the date_diff with the original DataFrame
df = pd.merge(df, date_diff, on='user_id')

# Step 4: Rename the newly added column
df.rename(columns={'date_y': 'review_date_diff'}, inplace=True)



In [54]:
df.to_csv('amazon_reviews2.csv')

In [23]:
#CHECK FOR DUPLICATE REVIEW TEXTS USING JACCARD SIMILARITY
from nltk import ngrams
import spacy

en = spacy.load('en_core_web_sm')
stop_words= en.Defaults.stop_words

def jaccard_similarity_score(review1, review2):

    # Convert review1 into set of unigrams without stop words
    set1 = set(ngrams([word for word in review1.lower().split()
               if word.lower() not in stop_words], n=1))
    # Convert review2 into set of unigrams without stop words
    set2 = set(ngrams([word for word in review2.lower().split()
               if word.lower() not in stop_words], n=1))
    intersection = len(set1.intersection(set2))  # Calculate intersection
    union = len(set1.union(set2))  # Calculate union
    similarity = intersection / union  # Calculate Jaccard similarity
    return similarity

##FINDS ALL PAIRS OF DUPLICATES
def find_duplicates(reviews):
    duplicates = []
    
    for i in range(len(reviews)):
        for j in range(i+1, len(reviews)):
            similarity = jaccard_similarity_score(reviews[i],reviews[j])
            if similarity > 0.9:
                duplicates.append((reviews[i], reviews[j]))
                
    return duplicates

##FINDS ALL REVIEWS HAVING ANOTHER DUPLICATE
def find_duplicate_reviews(df, threshold=0.9):
    duplicates = []
    reviewed = set()
    
    for i, row in df.iterrows():
        review = row['review']
        if review in reviewed:
            continue
        
        reviewed.add(review)
        
        for j, other_row in df[i+1:].iterrows():
            other_review = other_row['review']
            similarity = jaccard_similarity_score(review, other_review)
            
            if similarity > threshold:
                duplicates.append((review, other_review))
    
    return duplicates


In [ ]:
find_duplicate_reviews(df)